# 웹 스크래핑 프로그램을 만들어보자
1. 예제 페이지에서 여행기사를 추출한다.   
https://korean.visitkorea.or.kr/main/main.do
2. 검색어와 스크래핑 건수를 입력받는다. (input)
3. 페이지를 이동한다.
4. 상세 페이지별로 여행기사 정보를 수집한다.
5. 이미지와 텍스트를 추출해서 파일로 저장한다. 
- 6. pyinstaller로 .exe 빌드

## 코드 작성 로직

- 작은구조를 먼저 만들고 점차 큰 구조를 단계적으로 만들어 간다
- (1) 페이지 이동 
- (2) 페이지 내 연동된 '상세페이지 들어가기' 
- (3) 상세페이지 내용 수집하기
- (4) 페이지로 돌아오기(=나오기 : driver.back()) 
- => 페이지 이동할 때 마다 한 페이지 내용을 수집하는 함수를 넣으면 되겠구나
- 페이지 워킹은 코드를 먼저 만들고 -> 이걸 함수화 하자
---------------------------------------------------------------------
- 수집 과정 각각의 단계를 함수로 만들기
- 그 다음에 프로그램용 코드는 함수 위주로 조합

## 될 때까지 다시 도전 ㅠㅠ

In [14]:
# 필요한 모듈 호출
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math, time   

from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time, urllib.request
import os

In [15]:
# 검색어(여행지, 행사명 등) 입력
search = input('검색어를 입력하세요.')

# 크롤링 건수 입력/ 추출 페이지 수 계산
cnt = int(input('크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10) # 크롤링 할 전체 페이지 수(올림) # 이 페이지는 페이지당 10건이 최대
print(page_cnt)

검색어를 입력하세요.강릉
크롤링 할 건수는 몇건입니까?: 22
3


In [16]:
# URL 지정/웹드라이버 로드
URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(executable_path="C:/Users/Jiyoung/Documents/workspace/chromedriver.exe")
driver.get(URL)
time.sleep(3)

C:\Users\Jiyoung\AppData\Local\Temp\ipykernel_15832\3750613393.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/Jiyoung/Documents/workspace/chromedriver.exe")


In [17]:
# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()

In [18]:
# 제목 추출
# 한 페이지당 10개의 게시물이 나오므로 len(result)는 10이 나와야 하는데...?

result = driver.find_elements(By.CSS_SELECTOR, '.tit a')
len(result)

29

In [19]:
# for 문으로 1페이지 수집해서 상태를 확인하자

for i in result:
    print(i.text)

# 앞 부분에 이 수많은 여백은 뭐지??




















우리가 몰랐던 강릉 - 한옥과 경포호에 깃든 이 고장의 매력을 찾아서
각양각색의 한옥들이 모여있는 강릉오죽한옥마을
오감이 즐거운 강릉 여행 코스
[더가게 함께가게] 바다에서 시작해 바다로 끝나는 강릉 당일치기 코스
낮과 밤, 모두 즐겁다! 강릉솔향수목원
강원도 여행, 시원한 솔숲부터 푸른 바다까지 즐기는 강릉 여행 코스
강릉 여행을 추억하는 특별한 방법, 강릉 리고엠(Ligo-em)
가시연습지에 가시연꽃이 피었습니다, 강릉 가시연습지
<더글로리>, <도깨비> 김은숙 작가의 드라마 속 세상, 강릉으로 가다.
[워크앤스테이 강릉] 푸른 강릉 바다에 위치한 전문 코워킹 스페이스


In [20]:
# 이 상태로는 안되니까 기사 제목을 더 정확하게 찾아줘야겠다

result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
len(result)

10

In [21]:
# 한 페이지 단위로 콘텐츠별 상세페이지 정보 추출

from bs4 import BeautifulSoup
contents_no = 0

for item in result:
    contents_no += 1
    item.click()
    time.sleep(3)  
    
    print(f'======= [ {contents_no} ]  =======')
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')
    title = soup.find(id='topTitle')
    print(title.text)

    driver.back()
    time.sleep(3)

print('==== 완료 ====')

======= [ 1 ]  =======
우리가 몰랐던 강릉 - 한옥과 경포호에 깃든 이 고장의 매력을 찾아서
======= [ 2 ]  =======
각양각색의 한옥들이 모여있는 강릉오죽한옥마을


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:goSearchDetail('e70371ac-424a-43bc-9800-c2266bf5004a');">...</a> is not clickable at point (100, 583). Other element would receive the click: <img class="_sa_img" src="https://image.tand.kr/msg/wkjGTTa4gvoXNE52Ae9j/campaign-KTO-ebac3-5CuyRRUDnjHk7azc9_K1/hw2-iHmhcgovsfnBTYXo.png" alt="[페이지뷰] 첫방문자 안착유도) 대구석/첫방문자 A-1">
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF6F7044D02+56194]
	(No symbol) [0x00007FF6F6FB04B2]
	(No symbol) [0x00007FF6F6E576AA]
	(No symbol) [0x00007FF6F6EA8763]
	(No symbol) [0x00007FF6F6EA675D]
	(No symbol) [0x00007FF6F6EA3F5C]
	(No symbol) [0x00007FF6F6EA2D15]
	(No symbol) [0x00007FF6F6E9687F]
	(No symbol) [0x00007FF6F6EC5E7A]
	(No symbol) [0x00007FF6F6E96226]
	(No symbol) [0x00007FF6F6EC6320]
	(No symbol) [0x00007FF6F6EE2786]
	(No symbol) [0x00007FF6F6EC5C23]
	(No symbol) [0x00007FF6F6E94A45]
	(No symbol) [0x00007FF6F6E95AD4]
	GetHandle

In [38]:
# 페이지 이동하며 기사 타이틀 수집

tit_xpath = '//*[@id="search_result"]/ul/li[*]/div[1]/div[1]/a'
no = 0

for x in range(1, page_cnt+1):  # 페이지에 대한 반복문이 먼저 돌아야 함
    print(f'========= {x} 페이지 작업 =========')
    mylist = driver.find_elements(By.XPATH, tit_xpath)
    
    for item in mylist:  # 페이지 안에서 데이터 수집을 위한 반복문 돌기
        no += 1   # 개수를 하나씩 세어줌 (enumerate 써도 됨)
        if no > cnt:         # cnt에 설정한 건수 보다 높으면 break
            break
        print(no, item.text.replace('\n', ''))
    
    if no <= cnt:   # 첫번째 페이지는 이미 내가 들어가서 코드를 실행하니까 
                    # 페이지 클릭은 전체 과정의 마지막 단계에서 실행
        a = f'/html/body/div[3]/div/div[1]/div[15]/a[{x+1}]'
        driver.find_element(By.XPATH, a).click()
        time.sleep(2)

print('========= 작업 완료 =========')



========= 1 페이지 작업 =========
1 오감이 즐거운 강릉 여행 코스
2 별 하나, 나 하나. 외갓집처럼 편안한 팜크닉, 평창 오대산힐링빌리지
3 꽃불 흩날리는 춘야(春夜)의 낙화(落火)쇼! 함안낙화놀이
4 봄꽃이 가득한 제주 나들이
5 밤이 되면 깨어나는 마법의 세계, 대전 오월드 나이트유니버스
6 빛으로 물든 '로맨틱 제주' <제주 야간 관광지>
7 다양한 축제와 어트랙션, 동물과 식물을 함께 만나는 글로벌 테마파크 - 용인 에버랜드
8 낭만과 그리움을 찾아서, 인천 경인아라뱃길 정서진 드라이브
9 오감으로 숲을 느끼는 쉼, 국립칠곡숲체원
10 2023 한국관광홍보영상에 등장하는 여행지를 만나보세요! Feel the Rhythm of Korea
========= 2 페이지 작업 =========
11 [구석이가 추천하는 축제여행 2탄] 국화향 따라 떠나는 남쪽나라 여행, 마산가고파국화축제
12 흥미진진 유쾌상쾌축제가 있어 더 즐거운 내나라 여행
13 추위에 맞서 겨울을 제대로 즐기는 방법 '가평 청평얼음꽃송어축제'
14 소이산 모노레일 타고 황금들판 구경갈래?
15 봄 여행! 기차, 버스 말고, 4色 유람선
16 DMZ 10월 캘린더
17 빌딩 숲 어우러진 호숫가 산책, 수원 광교호수공원
18 다 같이 돌자! 섬 속의 섬, 우도 한 바퀴
19 청정 자연에서 즐기는 힐링 축제
20 잠실야구장 꿀팁 궁금하지 않아?야구장 초보들을 위한 가이드
========= 3 페이지 작업 =========
21 “고마, 퍼뜩 오이소~” 오감이 즐거운 4월의 부산 축제
22 블루로드 따라 떠나는 영덕 1박2일 여행
23 풍경을 훔치는 식탁, 눈으로 맛으로 분위기로 즐기는 시월의 한 끼
24 세계적인 명소에서 맞는 신년 해맞이, 제주 성산일출축제
25 덕유산 능선 따라 야생화 트레킹
26 어디 갈지 고민돼? 설레는 크리스마스 이거면 돼!
27 기암·항구·해변이 멋진 여름 바다로 떠나다, 주문진 아들바위
28 호수에 어린 오색빛 꿈길여행
29 [여행 카드] 천 년

In [22]:
driver.close()

In [39]:
# 내용 추출 (샘플)
contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
print(contents[0].text)

IndexError: list index out of range

In [ ]:
# 내용 추출 (이건 아직 인덱스 단위로 나뉘어져 있다=리스트)
contents = driver.find_elements(By.CLASS_NAME, 'txt_p')

contents_list = []
for item in contents:
    contents_list.append(item.text)
contents_list

In [ ]:
# for문으로 수집한 리스트를 하나의 문자열로 통합
contents_merge = ' '.join(contents_list) 
contents_merge

In [60]:
# 페이지 이동하며 기사 타이틀 수집


tit_xpath = '//*[@id="search_result"]/ul/li[*]/div[1]/div[1]/a'
no = 0

for x in range(1, page_cnt+1):  # 페이지에 대한 반복문이 먼저 돌아야 함
    print(f'========= {x} 페이지 작업 =========')
    mylist = driver.find_elements(By.XPATH, tit_xpath)
    contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
    
    for item in mylist:  # 페이지 안에서 데이터 수집을 위한 반복문 돌기
        # 내용 추출 (이건 아직 인덱스 단위로 나뉘어져 있다=리스트)
        contents_list = []
        for item in contents:
            contents_list.append(item.text)
            
            no += 1   # 개수를 하나씩 세어줌 (enumerate 써도 됨)
            if no > cnt:         # cnt에 설정한 건수 보다 높으면 break
                break
            print(no, item.text.replace('\n', ''))
    
        if no <= cnt:   # 첫번째 페이지는 이미 내가 들어가서 코드를 실행하니까 
                    # 페이지 클릭은 전체 과정의 마지막 단계에서 실행
            a = f'/html/body/div[3]/div/div[1]/div[15]/a[{x+1}]'
            driver.find_element(By.XPATH, a).click()
            time.sleep(2)

print('========= 작업 완료 =========')


========= 1 페이지 작업 =========
========= 2 페이지 작업 =========
========= 3 페이지 작업 =========
========= 작업 완료 =========


In [20]:
# 이미지 스크래핑

from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time, urllib.request
import os
f_dir = input('이미지 저장 폴더명 : ')

now = time.localtime()
s = '%04d%02d%02d_%02d%02d%02d'%(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
f_dir = f_dir +'_' + s
os.makedirs(os.getcwd()+'\\' + f_dir) # 이 경로에 폴더를 만들래

# 이미지 추출에 앞서 미리 스크롤을 끝까지 내려놔야 이미지를 추출할 수 있다
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

html_src = driver.page_source
html_dom = BeautifulSoup(html_src, 'lxml')

# 페이지 내에 있는 모든 이미지 주소 뽑기
mylist = html_dom.select('.img_typeBox img')
img_list = [item['src'] for item in mylist]

# urllib 활용해서 이미지 수집
no = 0
for src in img_list:
    # 다운로드  (주소, 파일이름)
    urllib.request.urlretrieve(src, f'{f_dir}\\{no}.jpg')
    no += 1

이미지 저장 폴더명 : 불꽃


In [32]:
driver.close()

MaxRetryError: HTTPConnectionPool(host='localhost', port=55924): Max retries exceeded with url: /session/40b271a7a229a53ce15e5a0479d5f58f/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FA350A3580>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [61]:
driver.quit()